In [ ]:
%matplotlib inline
from pylab import *
import os
import sys
from importlib import reload
import glob
import pickle
import time

# Specific science modules
import healpy as hp
import numpy as np

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import qubic
from qubic import SpectroImLib as si
from qubic import QubicSkySim as qss
from qubic import NamasterLib as nam
from qubic import camb_interface as qc
from qubic import fibtools as ft
from qubic.utils import progress_bar

rc('figure', figsize=(13, 6))
rc('font', size=15)

In [ ]:
### Initialize
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

dictfilename = global_dir + '/dicts/BmodesNoDustNoSystPaper0_2020.dict'


# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside']=256


In [ ]:
##### Function to get real end-2-end NERSC simulations from Louise
def get_maps_from_louise(directory, nfsub, config):
    residuals_patch = np.load(directory + f'residuals_{nfsub}bands_{config}.npy')
    seenmap = np.load(directory + f'seenmap_{nfsub}bands_{config}.npy')
    coverage_patch = np.load(directory + f'coverage_{nfsub}bands_{config}.npy')
    nside = 256
    residuals = np.zeros((nfsub, 12 * nside ** 2, 3))
    residuals[:, seenmap, :] = residuals_patch
    coverage = np.zeros(12 * nside ** 2)
    coverage[seenmap] = coverage_patch
    return residuals, coverage, seenmap


In [ ]:
nsub = 2
simname = 'FI150'
directory = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/MapsLouiseAtmosphereOK_Sept7_2020/'
residuals_maps, coverage, seen = get_maps_from_louise(directory, nsub, simname)



print(np.shape(residuals_maps))
center = [0,0]

rc('figure', figsize=(16, 10))
res = 15
figure()
st = ['I', 'Q', 'U']
rng = [20, 20, 20]
for i in range(nsub):
    for s in range(3):
        hp.gnomview(residuals_maps[i,:,s], rot=center, reso=res, sub=(nsub,3,3*i+s+1), 
                    title=simname+' '+st[s]+' Sub = {}/{}'.format(i+1,nsub),
                   min=-rng[s], max=rng[s], unit=r'$\mu K$')
tight_layout()

In [ ]:
##### Function ti get FastSiulator maps
def get_map_band(band, qubic_sky = None, duration=3., nsnew = None, spatial_noise=False, old_config=False, nbands=1):
    dictfilename = global_dir + '/dicts/pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictfilename)
    d['nside'] = 256
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
    d['nf_recon'] = nbands
    d['nf_sub'] = nbands

    # Frequency band
    dA = d.copy()
    dA['filter_nu'] = band * 1e9

    seed = np.random.randint(100000)
    sky_config = {'cmb': seed}

    if qubic_sky is None:
        Qubic_sky_A = qss.Qubic_sky(sky_config, dA)
    else:
        Qubic_sky_A = qubic_sky

    ##### Getting FastSimulator output maps
    maps_qubicA, maps_noiselessA, maps_noiseA, coverageA = Qubic_sky_A.get_partial_sky_maps_withnoise(spatial_noise=spatial_noise,
                                                                                 Nyears=duration, old_config=old_config)

    if nsnew is not None:
        print('Changinf to nside={}'.format(nsnew))
        sh = np.shape(maps_qubicA)
        new_maps = np.zeros((sh[0], 12*nsnew**2, 3))
        for i in range(sh[0]):
            for j in range(3):
                new_maps[i,:,j] = hp.ud_grade(maps_noiseA[i,:,j], nsnew)
        maps_noiseA = new_maps
        coverageA = hp.ud_grade(coverageA, nsnew, power=-2)
    
    return maps_noiseA, coverageA, Qubic_sky_A

In [ ]:
spatial_noise=False
nsub = 5

#### Figure with error for each band...
band = 150
duration = 2.
maps_noise, coverage, qubic_sky = get_map_band(band, duration=duration, nbands=nsub, 
                                                spatial_noise=spatial_noise)


print(np.shape(residuals_maps))
center = [0,0]

rc('figure', figsize=(16, 10))
res = 15
figure()
st = ['I', 'Q', 'U']
rng = [5, 5, 5]
for i in range(nsub):
    for s in range(3):
        hp.gnomview(maps_noise[i,:,s], rot=center, reso=res, sub=(nsub,3,3*i+s+1), 
                    title=simname+' '+st[s]+' Sub = {}/{}'.format(i+1,nsub),
                   min=-rng[s], max=rng[s], unit=r'$\mu K$')
tight_layout()

# Now compare

In [ ]:
def get_iqurms(maps, nbins=30, separate=True, center=[0,0], label='', sign='o', ls=':'):
    sh = np.shape(maps)
    nsub = sh[0]
    Irms = np.zeros(nsub)
    QUrms = np.zeros(nsub)
    ymax = 50
    for s in range(nsub):
        subplot(1,2,1)
        xx, yyI, yyQ, yyU = qss.get_angular_profile(maps[s,:,:], nbins=30, separate=True, center=[0,0])
        yyQU = 0.5*(yyQ + yyU)
        limit = 10.
        meanvalI = np.mean(yyI[xx<limit])# * pix_size
        meanvalQU = np.mean(yyQU[xx<limit])# * pix_size
        Irms[s] = meanvalI
        QUrms[s] = meanvalQU
        subplot(1,2,1)
        p1=plot(xx, yyI, sign, ls=ls)
        axhline(y=meanvalI, label='I '+label+r' {0:}/{1:} RMS = {2:5.1f} $\mu K$'.format(s, nsub, meanvalI), color=p1[0].get_color(), ls=ls)
        xlim(0,20)
        ylim(0,ymax)
        xlabel('Angle [deg]')
        ylabel('Pixels RMS [$\mu K$]')
        title('Nsub = {}'.format(nsub))
        legend()
        subplot(1,2,2)
        p1=plot(xx, yyQU, sign, color=p1[0].get_color(), ls=ls)
        xlim(0,20)
        ylim(0,ymax)
        axhline(y=meanvalQU, label='QU '+label+r' {0:}/{1:} RMS = {2:5.1f} $\mu K$'.format(s, nsub,meanvalQU), color=p1[0].get_color(), ls=ls)
        title('Nsub = {}'.format(nsub))
        legend()
    return Irms, QUrms
    


In [ ]:
nsubs = [1,2,3,4,5,8]
freqs = [150, 220]
duration = 2
pix_size = hp.nside2resol(nsnew, arcmin=True)
all_Irms_e2e = []
all_QUrms_e2e = []
all_Irms_fs = []
all_QUrms_fs = []
myfreq = []
for nsub in nsubs:
    for freq in freqs:
        figure()
        ### E2E maps
        simname = 'FI{}'.format(freq)
        maps_e2e, coverage, seen = get_maps_from_louise(directory, nsub, simname)
        Irms_e2e, QUrms_e2e = get_iqurms(maps_e2e, label='E2E {} '.format(freq), sign='o', ls=':')
        all_Irms_e2e.append(Irms_e2e)
        all_QUrms_e2e.append(QUrms_e2e)

        ### FastSIm maps
        maps_fs, coverage, qubic_sky = get_map_band(freq, duration=duration, nbands=nsub, 
                                                    spatial_noise=spatial_noise)
        Irms_fs, QUrms_fs = get_iqurms(maps_fs, label='FastSim {} '.format(freq), sign='x', ls='--')
        all_Irms_fs.append(Irms_fs)
        all_QUrms_fs.append(QUrms_fs)
        p1 = None
        p2 = None
        myfreq.append(freq)
    show()

    

In [ ]:
subplot(2,2,1)
for i in range(len(nsubs)):
    nsub = nsubs[i]
    xx = arange(nsub)
    print(xx)
    p=plot(range(nsub), all_Irms_e2e[2*i], 'o-',label = 'I e2e {}'.format(nsub))
    p=plot(range(nsub), all_Irms_fs[2*i], 'x--',label = 'I fs {}'.format(nsub), color=p[0].get_color())
    xlabel('# sub-bands')
    ylabel(r'RMS for $\theta<10$ deg.')
legend(fontsize=12)
title('I 150 GHz')

subplot(2,2,2)
for i in range(len(nsubs)):
    nsub = nsubs[i]
    xx = arange(nsub)
    print(xx)
    p=plot(range(nsub), all_QUrms_e2e[2*i], 'o-',label = 'I e2e {}'.format(nsub))
    p=plot(range(nsub), all_QUrms_fs[2*i], 'x--',label = 'I fs {}'.format(nsub), color=p[0].get_color())
    xlabel('# sub-bands')
    ylabel(r'RMS for $\theta<10$ deg.')
legend(fontsize=12)
title('QU 150 GHz')

subplot(2,2,3)
for i in range(len(nsubs)):
    nsub = nsubs[i]
    xx = arange(nsub)
    print(xx)
    p=plot(range(nsub), all_Irms_e2e[2*i+1], 'o-',label = 'I e2e {}'.format(nsub))
    p=plot(range(nsub), all_Irms_fs[2*i+1], 'x--',label = 'I fs {}'.format(nsub), color=p[0].get_color())
    xlabel('# sub-bands')
    ylabel(r'RMS for $\theta<10$ deg.')
legend(fontsize=12)
title('I 220 GHz')

subplot(2,2,4)
for i in range(len(nsubs)):
    nsub = nsubs[i]
    xx = arange(nsub)
    print(xx)
    p=plot(range(nsub), all_QUrms_e2e[2*i+1], 'o-',label = 'I e2e {}'.format(nsub))
    p=plot(range(nsub), all_QUrms_fs[2*i+1], 'x--',label = 'I fs {}'.format(nsub), color=p[0].get_color())
    xlabel('# sub-bands')
    ylabel(r'RMS for $\theta<10$ deg.')
legend(fontsize=12)
title('QU 220 GHz')

tight_layout()

In [ ]:
len(myfreq)

In [ ]:
len(all_QUrms_e2e)